## Recommendation System Based on  Actor (Actor's Name as User Input)

### Importing the Required Libraries & Files

In [1]:
import pandas as pd
import numpy as np
import sys

movies= pd.read_csv("movies.dat",encoding='latin-1',sep='\t')
movies.drop(['imdbID','spanishTitle','rtID','rtID', 'rtAllCriticsRating', 'rtAllCriticsNumReviews',\
       'rtAllCriticsNumFresh', 'rtAllCriticsNumRotten', 'rtAllCriticsScore',\
       'rtTopCriticsRating', 'rtTopCriticsNumReviews', 'rtTopCriticsNumFresh',\
       'rtTopCriticsNumRotten', 'rtTopCriticsScore',\
       'rtAudienceNumRatings', 'rtAudienceScore', 'rtPictureURL'],axis=1,inplace=True)
movies['title'].loc[0]='Toy Story'

movie_genre= pd.read_csv("movie_genres.dat",encoding='latin-1',sep='\t')
movie_genre = movie_genre.groupby('movieID').agg({ 'genre': ', '.join}).reset_index()

movie_director= pd.read_csv('movie_directors.dat',encoding='latin-1',sep='\t')

movie_director.drop(['directorID'],axis=1,inplace=True)

movie_actor= pd.read_csv('movie_actors.dat',encoding='latin-1',sep='\t')
movie_actor.drop(['actorID'],axis=1,inplace=True)

merge1= pd.merge(movies,movie_genre,left_on='id',right_on='movieID')
merge2= pd.merge(merge1,movie_director)
merge3= pd.merge(merge2,movie_actor)
merge3.drop(['movieID'],axis=1,inplace=True)

user_ratedmovies= pd.read_csv('user_ratedmovies.dat',encoding='latin-1',sep='\t')
user_ratedmovies.drop(['date_day', 'date_month', 'date_year',\
       'date_hour', 'date_minute', 'date_second'],axis=1,inplace=True)

final_data= pd.merge(merge3,user_ratedmovies,left_on='id',right_on='movieID')

C:\Users\Arpitha Ananth\Anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


### Taking Actor's Name as User Input

In [2]:
actor_needed= str(input("Enter the Full Name of the Actor: "))

Enter the Full Name of the Actor: Bill Farmer


In [3]:
### Finding all the Movies where the actor has played the Lead Role
actor_mov= movie_actor[movie_actor['actorName']==actor_needed]

### Checking if Actor is present in the database or not
if len(actor_mov)>=1:
    print("Actor Found")
else:
    print("There is no Actor by this Name, please re-check")
    sys.exit("Actor Not Found in Database")

Actor Found


### Finding all the movies which has been watched by the User 

In [4]:
user_ratedmovies.groupby(['userID','movieID']).count()
user= int(input("Enter your User ID : "))

Enter your User ID : 78


In [5]:
subset_user_ratedmovies= user_ratedmovies[user_ratedmovies['userID']==user]

### Checking if User is present in the database or not
if len(subset_user_ratedmovies)>=1:
    print("User Found")
else:
    print("Invalid User ID, please re-check")
    sys.exit("User Not Found In Database")

User Found


In [6]:
### Recommending Movies of the Actor which has not been watched by User Yet

df_all = actor_mov.merge(subset_user_ratedmovies.drop_duplicates(), on=['movieID'],how='left', indicator=True)
recc1= df_all[df_all['_merge']=='left_only']
recc_movie= recc1[['movieID','actorName']]
final_recc= pd.merge(recc_movie,movies,left_on='movieID',right_on='id',how='left')
final_recc.drop(['id'],inplace=True,axis=1)

### List of Movies of the Actor which has not been watched by User Yet
final_recc= final_recc.sort_values(by='rtAudienceRating',ascending=False)
final_recc.drop_duplicates('title',inplace=True,keep='first')

###Top 5  Movie Recommendations
final_recc.head(5)

,movieID,actorName,title,imdbPictureURL,year,rtAudienceRating
0,1,Bill Farmer,Toy Story,http://ia.media-imdb.com/images/M/MV5BMTMwNDU0...,1995,3.7
6,45517,Bill Farmer,Cars,http://ia.media-imdb.com/images/M/MV5BMTQ1NTIx...,2006,3.7
7,58299,Bill Farmer,Horton Hears a Who!,http://ia.media-imdb.com/images/M/MV5BMTI3NDA3...,2008,3.7
1,239,Bill Farmer,A Goofy Movie,http://ia.media-imdb.com/images/M/MV5BMTI0Mjgz...,1995,3.1
2,673,Bill Farmer,Space Jam,http://ia.media-imdb.com/images/M/MV5BMTQ5NDg1...,1996,2.9
